In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import linregress

In [5]:
pdf = pd.read_csv('data/Patient_meta.csv')
sdf = pd.read_csv('data/Study_results.csv')


In [6]:
sdf.rename(columns={'Mouse ID':'Patient_ID'}, inplace=True)
sdf

,Patient_ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.000000,0
1,f932,0,45.000000,0
2,g107,0,45.000000,0
3,a457,0,45.000000,0
4,c819,0,45.000000,0
...,...,...,...,...
1888,r944,45,41.581521,2
1889,u364,45,31.023923,3
1890,p438,45,61.433892,1
1891,x773,45,58.634971,4


In [9]:
mdf = pd.merge(pdf, sdf, on='Patient_ID', how='left')
mdf.head()

,row,Patient_ID,DrugRegimen,Sex,Age_months,Weight,Name,Marital Status,Ethnicity,Address,ContactNumbers,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",0,45.000000,0
1,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",5,38.825898,0
2,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",10,35.014271,1
3,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",15,34.223992,1
4,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",20,32.997729,1


In [38]:
mdf.Name = '[Name]'
mdf.Address = '[Address]'
mdf.ContactNumbers = '[ContactNumber]'
mdf

,row,Patient_ID,DrugRegimen,Sex,Age_months,Weight,Name,Marital Status,Ethnicity,Address,ContactNumbers,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,0,k403,Ramicane,Male,21,16,[Name],D,Asian,[Address],[ContactNumber],0,45.000000,0
1,0,k403,Ramicane,Male,21,16,[Name],D,Asian,[Address],[ContactNumber],5,38.825898,0
2,0,k403,Ramicane,Male,21,16,[Name],D,Asian,[Address],[ContactNumber],10,35.014271,1
3,0,k403,Ramicane,Male,21,16,[Name],D,Asian,[Address],[ContactNumber],15,34.223992,1
4,0,k403,Ramicane,Male,21,16,[Name],D,Asian,[Address],[ContactNumber],20,32.997729,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,248,z969,Naftisol,Male,9,30,[Name],S,Cauc,[Address],[ContactNumber],25,63.145652,2
1889,248,z969,Naftisol,Male,9,30,[Name],S,Cauc,[Address],[ContactNumber],30,65.841013,3
1890,248,z969,Naftisol,Male,9,30,[Name],S,Cauc,[Address],[ContactNumber],35,69.176246,4
1891,248,z969,Naftisol,Male,9,30,[Name],S,Cauc,[Address],[ContactNumber],40,70.314904,4


In [13]:
len(set(mdf['Patient_ID']))

249

In [28]:
filt = mdf.duplicated(subset=['Patient_ID', 'Timepoint'])
df_dup = mdf.loc[filt]
df_dup

,row,Patient_ID,DrugRegimen,Sex,Age_months,Weight,Name,Marital Status,Ethnicity,Address,ContactNumbers,Timepoint,Tumor Volume (mm3),Metastatic Sites
909,107,g989,Propriva,Female,21,26,Shauna Drayton,D,Cauc,123 Elm ST. Wingate TX 79566,"p: 261-1226, c: 326-3412, fax: 926-5555",0,45.000000,0
911,107,g989,Propriva,Female,21,26,Shauna Drayton,D,Cauc,123 Elm ST. Wingate TX 79566,"p: 261-1226, c: 326-3412, fax: 926-5555",5,47.570392,0
913,107,g989,Propriva,Female,21,26,Shauna Drayton,D,Cauc,123 Elm ST. Wingate TX 79566,"p: 261-1226, c: 326-3412, fax: 926-5555",10,49.880528,0
915,107,g989,Propriva,Female,21,26,Shauna Drayton,D,Cauc,123 Elm ST. Wingate TX 79566,"p: 261-1226, c: 326-3412, fax: 926-5555",15,53.442020,0
917,107,g989,Propriva,Female,21,26,Shauna Drayton,D,Cauc,123 Elm ST. Wingate TX 79566,"p: 261-1226, c: 326-3412, fax: 926-5555",20,54.657650,1


In [31]:
pid = df_dup['Patient_ID']
did = set(pid)
did

{'g989'}

In [33]:
clean_df = mdf.loc[~mdf.Patient_ID.isin(did)]
clean_df

,row,Patient_ID,DrugRegimen,Sex,Age_months,Weight,Name,Marital Status,Ethnicity,Address,ContactNumbers,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",0,45.000000,0
1,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",5,38.825898,0
2,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",10,35.014271,1
3,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",15,34.223992,1
4,0,k403,Ramicane,Male,21,16,Arlen Marshall,D,Asian,123 Elm ST. Sterling VA 20165,"p: 161-1216, c: 316-3412, fax: 916-5555",20,32.997729,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,248,z969,Naftisol,Male,9,30,Dottie Call,S,Cauc,123 Elm ST. Alden NY 14004,"p: 301-1230, c: 330-3412, fax: 930-5555",25,63.145652,2
1889,248,z969,Naftisol,Male,9,30,Dottie Call,S,Cauc,123 Elm ST. Alden NY 14004,"p: 301-1230, c: 330-3412, fax: 930-5555",30,65.841013,3
1890,248,z969,Naftisol,Male,9,30,Dottie Call,S,Cauc,123 Elm ST. Alden NY 14004,"p: 301-1230, c: 330-3412, fax: 930-5555",35,69.176246,4
1891,248,z969,Naftisol,Male,9,30,Dottie Call,S,Cauc,123 Elm ST. Alden NY 14004,"p: 301-1230, c: 330-3412, fax: 930-5555",40,70.314904,4


In [34]:
count = len(set(clean_df['Patient_ID']))
count

248